# MathematicalProgram 튜터리얼
notebook 실행 방법은 [index](./index.ipynb)를 참조하자.


## 배경(Background)
많은 엔지니어링 문제는 수학적 최적화 문제로 공식화될 수 있으며 수치 솔버(numerical solvers)를 통해 해결할 수 있다. 일반적인 수학적 최적화 문제는 다음과 같이 공식화할 수 있다.
$\begin{aligned}
\begin{array}{rl}
                       \min_x \;  &  f(x)
   \\\text{subject to}  \;  &  x \in\mathcal{S}
   \end{array}
   \qquad
   \boxed{
         \begin{array}{ll}
      \text{The real-valued decision variable is}       &x\\
      \text{The real-valued cost function is}           &f(x)\\
      \text{The constraint set is}                      &\mathcal{S}\\
      \text{The optimal } x \text{ that minimizes the cost function is}  &x^*
      \end{array}
      }
\end{aligned}$

여기서 $x$는 실수 값의 결정 변수이고, $f(x)$는 실수 값의 *비용 함수*이며, $\mathcal{S}$는 $x$에 대한 제약 조건 집합이다. 우리의 목표는 제약 조건 집합 $\mathcal{S}$ 내에서 최적의 $x^*$를 찾는 것이며, 이는 $x^*$가 비용 함수 $f(x)$를 최소화하는 것이다.

예를 들어, 다음 최적화 문제는 $x \ge 1$라는 제약 조건 하에서 $x^3 + 2x + 1$을 최소화하는 $x$의 값을 결정한다.
$\begin{aligned}
\begin{array}{rl}
\min_x\;&x^3 + 2x + 1\\
\text{subject to}\;\;&x \ge 1
\end{array}
\quad
\boxed{
      \begin{array}{ll}
          \text{The real-valued decision variable is}         &  x\\
          \text{The real-valued cost function }f(x) \text{ is} &  x^3 + 2x + 1\\
          \text{The set }\mathcal{S} \text{ of constraints is}      &  x \ge 1\\
          \text{The value that minimizes the cost function is}    &  x^* = 1
   \end{array}
}
\end{aligned}$

일반적으로 최적화 문제의 해결 방법은 문제의 분류에 따라 다르다. 분류에는 선형 프로그래밍, 2차 프로그래밍, 혼합 정수 프로그래밍 등이 포함됩니다. 분류는 비용 함수 $f(x)$와 제약 조건 집합 $\mathcal{S}$의 속성에 따라 결정됩니다. 예를 들어, 비용 함수 $f(x)$가 $x$의 선형 함수이고 제약 조건 $\mathcal{S}$가 선형 집합 $\mathcal{S} = \{x | Ax\le b\}$이면 *선형 프로그래밍* 문제가 된다. 이 문제는 특정 솔버를 사용하여 효율적으로 해결할 수 있다.

각 범주의 최적화 문제에는 여러 솔버가 있지만 각 솔버에는 고유한 API와 데이터 구조가 있다. 사용자는 솔버를 전환할 때 종종 코드를 다시 작성해야 한다. 이러한 문제를 해결하기 위해 Drake는 *MathematicalProgram* 클래스를 통해 공통 API를 제공한다. 솔버별 코드를 피하는 것 외에도 제약 조건과 비용 함수를 symbolic 형태로 작성할 수 있습니다(코드를 더 읽기 쉽게 만든다.).
Drake의 MathematicalProgram은 MATLAB의 YALMIP: https://yalmip.github.io/ 또는 Julia의 JuMP: https://github.com/JuliaOpt/JuMP.jl과 유사하게 최적화 문제를 상징적으로 정의하고 다양한 솔버와 호환하는 공통 API를 제공한다.
<br> 주의: Drake는 다양한 [solvers](https://drake.mit.edu/doxygen_cxx/group__solvers.html)를 지원한다.
(일부는 오픈 소스이고 일부는 라이센스가 필요하다.)

Drake는 다음과 같은 종류의 최적화 문제를 공식화하고 해결할 수 있다.
* Linear programming
* Quadratic programming
* Second-order cone programming
* Nonlinear nonconvex programming
* Semidefinite programming
* Sum-of-squares programming
* Mixed-integer programming (mixed-integer linear programming, mixed-integer quadratic programming, mixed-integer second-order cone programming).
* Linear complementarity problem

이 튜토리얼에서는 Drake의 MathematicalProgram 클래스의 기본 개념을 소개한다. 더 복잡한 기능에 대한 자세한 내용은 문서 [하단 고급 튜터리얼](#Advanced-tutorials)에서 확인하실 수 있습니다.

## MathematicalProgram class 기본(Basics of MathematicalProgram class)
Drake의 MathematicalProgram 클래스는 최적화 문제의 수학적 공식을 포함합니다. 즉, 결정 변수 $x$, 비용 함수 $f(x)$ 및 제약 조건 집합 $\mathcal{S}$를 포함한다.

### MathematicalProgram 객체 초기화(Initialize a MathematicalProgram object)

 이 클래스를 초기화하려면 먼저 빈 MathematicalProgram을 다음과 같이 만든다.

In [ ]:
from pydrake.solvers import MathematicalProgram
import numpy as np
import matplotlib.pyplot as plt

# Create an empty MathematicalProgram named prog (with no decision variables, 
# constraints or cost function)
prog = MathematicalProgram()



### 결정 변수 추가하기(Adding decision variables)
아래에서 보는바와 같이, `NewContinuousVariables` 함수는 `prog`에 두 개의 새로운 연속형 결정 변수를 추가한다. 새로 추가된 변수는 numpy 배열 `x`로 반환된다.
<br><font size=-1>변수의 범위는 연속 집합(continuous set)이라 이진 변수가 이산값으로 0 혹은 1만을 가지는 것과는 다르다는 것에 주의하자.</font>

In [ ]:
x = prog.NewContinuousVariables(2)


*x*에 있는 변수의 기본 이름은 "x(0)"과 "x(1)"이다. 다음 줄은 `x`의 기본 이름과 유형을 인쇄하는 반면, 두 번째 줄은 기호 표현식 "1 + 2x[0] + 3x[1] + 4x[1]"을 인쇄한다.

In [ ]:
print(x)
print(1 + 2*x[0] + 3*x[1] + 4*x[1])

"dog(0)" 및 "dog(1)"이라는 이름의 두 변수로 구성된 배열 `y`를 만들려면 `NewContinuousVariables()`의 두 번째 인수로 이름 "dog"를 전달한다. 아래에는 `y`에 있는 두 변수와 `y`를 포함하는 symbolic 표현식의 출력도 보여준다.

In [ ]:
y = prog.NewContinuousVariables(2, "dog")
print(y)
print(y[0] + y[0] + y[1] * y[1] * y[1])

"A"라는 이름의 $3 \times 2$ 변수 행렬을 만들려면 다음 코드를 입력한다.

In [ ]:
var_matrix = prog.NewContinuousVariables(3, 2, "A")
print(var_matrix)

### 제약 추가하기(Adding constraints)
결정 변수에 제약 조건을 부과하는 방법에는 여러 가지가 있다. 이 튜토리얼에서는 몇 가지 간단한 예를 보여준다. 다른 유형의 제약 조건에 대해서는 이 문서의 [하단](#Advanced-tutorials)에 있는 링크를 참조하자.



#### AddConstraint
제약을 추가하는 가장 간단한 방법은 `MathematicalProgram.AddConstraint()`를 사용한다.

In [ ]:
# Add the constraint x(0) * x(1) = 1 to prog
prog.AddConstraint(x[0] * x[1] == 1)

`prog`로 부등식 제약 조건을 추가할 수 있다.

In [ ]:
prog.AddConstraint(x[0] >= 0)
prog.AddConstraint(x[0] - x[1] <= 0)

`prog`는 이러한 symbolic 부등식 제약 조건 표현을 자동으로 분석하여 모두 $x$에 대한 *선형* 제약 조건이라고 판단합니다.

### Adding Cost functions
복잡한 최적화 문제에서 전체 비용 함수 $f(x)$를 개별 비용 함수의 합으로 작성하는 것이 편리한 경우가 많다.

$\begin{aligned}
f(x) = \sum_i g_i(x)
\end{aligned}$


#### AddCost method.
개별 비용 함수 $g_i(x)$를 전체 비용 함수 f(x)에 추가하는 가장 간단한 방법은 `MathematicalProgram.AddCost()` 메서드를 사용하는 것이다.(아래에 표시된 것처럼)

In [ ]:
# Add a cost x(0)**2 + 3 to the total cost. Since prog doesn't have a cost before, now the total cost is x(0)**2 + 3
prog.AddCost(x[0] ** 2 + 3)

전체 비용 함수 $f(x)$에 또 다른 개별 비용 함수 $x(0) + x(1)$을 추가하려면 다음과 같이 간단히 `AddCost()` 메서드를 다시 호출한다.

In [ ]:
prog.AddCost(x[0] + x[1])

이제 총 비용 함수는 $x(0)^2 + x(0) + x(1) + 3$이 된다.

`prog`는 이러한 각 개별 비용 함수를 분석하여 $x(0) ^ 2 + 3$이 볼록한 2차 함수(convex quadratic function)이고 $x(0) + x(1)$이 $x$의 선형 함수임을 확인할 수 있다.

언제든지 LaTeX에서 프로그램을 렌더링할 수 있습니다:

In [ ]:
from IPython.display import display, Markdown
display(Markdown(prog.ToLatex()))

### 최적화 문제 해결(Solve the optimization problem)
모든 결정 변수, 제약 조건, 비용 함수를 `prog`에 추가했다면 이제 최적화 문제를 해결할 준비가 되었다.



#### 자동으로 solver 선택하기(Automatically choosing a solver)
최적화 문제를 해결하는 가장 간단한 방법은 `Solve()` 함수를 호출하는 것이다. Drake의 MathematicalProgram은 제약 조건과 비용 함수의 유형을 분석하고, 문제에 적합한 솔버를 선택하여 호출한다. `Solve()` 호출 결과는 반환 인자에 저장된다. 다음은 코드 예시이다.

In [ ]:
"""
Solves a simple optimization problem
       min x(0)^2 + x(1)^2
subject to x(0) + x(1) = 1
           x(0) <= x(1)
"""
from pydrake.solvers import Solve
# Set up the optimization problem.
prog = MathematicalProgram()
x = prog.NewContinuousVariables(2)
prog.AddConstraint(x[0] + x[1] == 1)
prog.AddConstraint(x[0] <= x[1])
prog.AddCost(x[0] **2 + x[1] ** 2)

# Now solve the optimization problem.
result = Solve(prog)

# print out the result.
print("Success? ", result.is_success())
# Print the solution to the decision variables.
print('x* = ', result.GetSolution(x))
# Print the optimal cost.
print('optimal cost = ', result.get_optimal_cost())
# Print the name of the solver that was called.
print('solver is: ', result.get_solver_id().name())

`Solve`의 반환 인자에서 최적화 결과를 검색할 수 있다. 예를 들어, 솔루션 $x^*$는 `result.GetSolution()`에서 검색하고 최적 비용은 `result.get_optimal_cost()`에서 검색한다.

일부 최적화 문제는 실현 불가능하다.(솔루션이 없음). 예를 들어 다음 코드 예제에서 `result.get_solution_result()`는 `kSolutionFound`를 보고하지 않는다.

In [ ]:
"""
An infeasible optimization problem.
"""
prog = MathematicalProgram()
x = prog.NewContinuousVariables(1)[0]
y = prog.NewContinuousVariables(1)[0]
prog.AddConstraint(x + y >= 1)
prog.AddConstraint(x + y <= 0)
prog.AddCost(x)

result = Solve(prog)
print("Success? ", result.is_success())
print(result.get_solution_result())

#### 수동으로 solver 선택하기(Manually choosing a solver)

Drake가 자동으로 솔버를 선택하는 대신 직접 솔버를 선택하려는 경우, 솔버를 명시적으로 인스턴스화하고 해당 솔버의 `Solve` 함수를 호출할 수 있다. 솔버를 인스턴스화하는 데는 두 가지 방법이 있다. 예를 들어, 오픈 소스 솔버 [IPOPT](https://github.com/coin-or/Ipopt)를 사용하여 문제를 해결하려는 경우 다음 두 가지 방법 중 하나를 사용하여 솔버를 인스턴스화할 수 있다.:
1. 가장 간단한 방법은 `solver = IpoptSolver()`를 호출하는 것이다.
2. 두 번째 방법은 지정된 솔버 ID를 사용하여 솔버를 생성하는 것이다. `solver = MakeSolver(IpoptSolver().solver_id())`

In [ ]:
"""
Demo on manually choosing a solver
Solves the problem
min x(0)
s.t x(0) + x(1) = 1
    0 <= x(1) <= 1
"""
from pydrake.solvers import IpoptSolver
prog = MathematicalProgram()
x = prog.NewContinuousVariables(2)
prog.AddConstraint(x[0] + x[1] == 1)
prog.AddConstraint(0 <= x[1])
prog.AddConstraint(x[1] <= 1)
prog.AddCost(x[0])

# Choose IPOPT as the solver.
# First instantiate an IPOPT solver.

solver = IpoptSolver()
# The initial guess is [1, 1]. The third argument is the options for Ipopt solver,
# and we set no solver options.
result = solver.Solve(prog, np.array([1, 1]), None)

print(result.get_solution_result())
print("x* = ", result.GetSolution(x))
print("Solver is ", result.get_solver_id().name())
print("Ipopt solver status: ", result.get_solver_details().status,
      ", meaning ", result.get_solver_details().ConvertStatusToString())


참고로, `solver.Solve()`는 세 가지 입력 인자를 필요로 한다: 최적화 프로그램 `prog`, 결정 변수 값의 초기 추측 (이 경우 `[1, 1]`), 그리고 솔버를 위한 선택적 설정 (이 경우 `None`으로 기본 IPOPT 설정을 사용함). 초기 추측이 없으면 `solver.Solve(prog)`를 호출할 수 있다. Drake는 기본 초기 추측 (값이 0인 벡터)을 선택하지만 이 초기 추측은 최적화를 위한 좋지 않은 시작점일 수 있다. 다음 예제 코드에서 알 수 있듯이, 기본 초기 추측을 사용하면 솔루션이 존재하더라도 (초기 추측 [1, 1]로 찾을 수 있음에도) 솔버가 솔루션을 찾지 못할 수 있다.

In [ ]:
from pydrake.solvers import MakeSolver
solver = MakeSolver(IpoptSolver().solver_id())
result = solver.Solve(prog)
print(result.get_solution_result())
print("x* = ", result.GetSolution(x))

또한, 어떤 솔버가 호출되었는지 알고 있다면 `result.get_solver_details()`를 호출하여 솔버별로 결과에 접근할 수 있다. 예를 들어, `IpoptSolverDetails`에는 IPOPT 솔버의 상태 코드인 `status` 필드가 포함되어 있으며 다음과 같이 이 정보에 접근할 수 있다.

In [ ]:
print("Ipopt solver status: ", result.get_solver_details().status,
      ", meaning ", result.get_solver_details().ConvertStatusToString())

각 솔버에는 고유한 세부 정보가 있다. `result.get_solver_details()`의 반환 값에 저장된 내용을 확인하려면 `FooSolverDetails` 클래스를 참조해야 한다. 예를 들어, IPOPT가 호출되었다면 `IpoptSolverDetails` 클래스를, OSQP 솔버가 호출되었다면 `OsqpSolverDetails` 클래스를 참조하는 방식이다.

### 초기 추측 사용하기(Using an initial guess)
비선형 최적화와 같은 일부 최적화 문제는 초기 추측(initial guess)이 필요하다. 이차 프로그래밍(quadratic programming), 혼합 정수 최적화(mixed-integer optimization) 등과 같은 다른 유형의 문제는 좋은 초기 추측이 제공되면 더 빠르게 해결될 수 있다. 사용자는 `Solve` 함수에서 입력 인수로 초기 추측을 제공할 수 있다. 초기 추측이 제공되지 않으면 Drake는 값이 0인 벡터를 초기 추측으로 사용한다.

아래 예제에서는 초기 추측이 문제의 결과에 영향을 미칠 수 있음을 보여준다. 사용자가 제공한 초기 추측이 없으면 솔버가 솔루션을 찾지 못할 수도 있다.

In [ ]:
from pydrake.solvers import IpoptSolver
prog = MathematicalProgram()
x = prog.NewContinuousVariables(2)
prog.AddConstraint(x[0]**2 + x[1]**2 == 100.)
prog.AddCost(x[0]**2-x[1]**2)
solver = IpoptSolver()
# The user doesn't provide an initial guess.
result = solver.Solve(prog, None, None)
print(f"Without a good initial guess, the result is {result.is_success()}")
print(f"solution {result.GetSolution(x)}")
# Pass an initial guess
result = solver.Solve(prog, [-5., 0.], None)
print(f"With a good initial guess, the result is {result.is_success()}")
print(f"solution {result.GetSolution(x)}")

초기 추측 설정에 대한 자세한 내용은 [Nonlinear program](./nonlinear_program.ipynb)의 `Setting the initial guess` 섹션을 참조하자.

## callback 추가(Add callback)
일부 솔버는 각 반복에서 callback 함수를 추가하는 것을 지원한다. callback의 한 가지 용도는 현재 반복에서 솔버의 진행 상황을 시각화하는 것이다. `MathematicalProgram`은 `AddVisualizationCallback` 함수를 통해 이 사용법을 지원하지만, 사용법은 시각화에만 국한되지 않고 callback 함수는 무엇이든 할 수 있다. 다음은 예시이다.

In [ ]:
# Visualize the solver progress in each iteration through a callback
# Find the closest point on a curve to a desired point.

fig = plt.figure()
curve_x = np.linspace(1, 10, 100)
ax = plt.gca()
ax.plot(curve_x, 9./curve_x)
ax.plot(-curve_x, -9./curve_x)
ax.plot(0, 0, 'o')
x_init = [4., 5.]
ax.plot(x_init[0], x_init[1], 'x', color='red')

def update(x):
    ax.plot(x[0], x[1], 'x', color='red')
    
prog = MathematicalProgram()
x = prog.NewContinuousVariables(2)
prog.AddConstraint(x[0] * x[1] == 9)
prog.AddCost(x[0]**2 + x[1]**2)
prog.AddVisualizationCallback(update, x)
result = Solve(prog, x_init)

## 고급 튜터리얼(Advanced tutorials)
[Setting solver parameters](./solver_parameters.ipynb)

[Updating costs and constraints (e.g. for efficient solving of many similar programs)](./updating_costs_and_constraints.ipynb)

[Debugging tips](./debug_mathematical_program.ipynb)

[Linear program](./linear_program.ipynb)

[Quadratic program](./quadratic_program.ipynb)

[Nonlinear program](./nonlinear_program.ipynb)

[Sum-of-squares optimization](./sum_of_squares_optimization.ipynb)